# Refinitiv Data Platform Library for Python
## Content - Chain - Streaming

This notebook demonstrates how a Price object and a Chain object can be mixed to receive streaming real-time data for contituents of a chain.  

#### Learn more

To learn more about the Refinitiv Data Platform Data Libraries just connect to the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [login](https://developers.refinitiv.com/iam/login) to the Refinitiv Developer Community portal you will get free access to a number of learning materials like [Quick Start guides](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/quick-start), [Tutorials](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/learning), [Documentation](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/docs) and much more.  

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.thomsonreuters.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

## Import the library and load credentials

Credentials used by this notebook are stored in the ./credentials.ipynb. Please edit ./credentials.ipynb to set your credentials and run the next cell to continue with this scripts

In [1]:
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Call **open_session(session_type)** to create and open a session to connect to the Refinitiv Data Platform directly (session_type="platform") or via Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a local realtime infrastructure (session_type="deployed").

session_type can be:
- "desktop" (Default value)
- "platform"
- "deployed"

In [2]:
open_session()

### Define a callback to receive streaming data

In [3]:
df = DataFrame()

def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)


def update_dataframe(streaming_prices, instrument_name, fields):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)

## Get Chain consistent

In [4]:
dow = pricing.chain.Definition(universe="0#.DJI").get_stream(session=session)

dow.open(with_updates=False)
dow.close()
dow_constituents = dow.get_constituents()
dow_constituents

['AAPL.OQ',
 'AMGN.OQ',
 'AXP.N',
 'BA.N',
 'CAT.N',
 'CRM.N',
 'CSCO.OQ',
 'CVX.N',
 'DIS.N',
 'DOW.N',
 'GS.N',
 'HD.N',
 'HON.OQ',
 'IBM.N',
 'INTC.OQ',
 'JNJ.N',
 'JPM.N',
 'KO.N',
 'MCD.N',
 'MMM.N',
 'MRK.N',
 'MSFT.OQ',
 'NKE.N',
 'PG.N',
 'TRV.N',
 'UNH.N',
 'V.N',
 'VZ.N',
 'WBA.OQ',
 'WMT.N']

## Create a Price

In [5]:
pricing_stream = pricing.Definition(universe=dow_constituents, fields=["BID", "ASK"]).get_stream()

pricing_stream.on_update(update_dataframe)
pricing_stream.on_complete(create_dataframe)

## Open Price

In [6]:
pricing_stream.open()

,BID,ASK
AAPL.OQ,127.17,127.2
AMGN.OQ,232.5,248.9
AXP.N,None,None
BA.N,None,None
CAT.N,None,None
CRM.N,None,None
CSCO.OQ,54.1,54.17
CVX.N,None,None
DIS.N,None,None
DOW.N,None,None


## Close the Price

In [7]:
pricing_stream.close()

<StreamState.Closed: 1>

## Close the session

In [8]:
close_session()